# <center >Version Finale : mini-projet 1 : L'IA pour les mesures des champs électromagnétiques </center>

Dans ce mini projet j'ai travaillé sur la daset "**ANFR_Herault_Mesures.csv**" , que j'ai téléchargé a partir de ce lien : https://data.montpellier3m.fr/dataset/antennes-relais-et-frequences-electromagnetiques-de-lherault

En fait les données de ce fichier renseigne les mesures des champs électromagnétiques sur le département de l'Hérault.

* Cette dataset comporte 16 colonnes, mais on va s'interessé au colonne "mesure globale" . 
* On remarque tout d'abord que les valeur varie entre 0v et 9v . Mais la majorité des valeures sont comprises entre 0v et 2v

* Observons en plus la colonne conformité, cette colonne indique la conformité ou non de l'antenne. et ceci se fait a travers la comparaison de la valeur du niveau global avec la valeur du seuil (6v).  
* La pertite partie théorique ci dessous va vous mieux expliquer cette notion de conformité:

## Partie théorique

### 1. Réglementation et protocoles de mesure

* Le décret n° 2002‐775 du 3 mai 2002 fixe les valeurs limites d’exposition du public aux
champs électromagnétiques (En particulier mesure du champ électrique). Ces valeurs
limites sont comprises entre 28 V/m et 87 V/m selon les fréquences.

la figure ci dessous présente les valeurs limites réglementaires fixées en France par le décret du mail 2001 n 2002-775

<img src="images/reglementation.PNG" />

* Toute personne physique ou morale peut faire mesurer l’exposition aux ondes
électromagnétiques aussi bien dans les locaux d’habitation que dans des lieux accessibles au
public (parcs, commerces...).

### 2. Protocoles de mesure

Dans le cadre de ses missions, l’Agence nationale des fréquences (ANFR en France et
ANF en Tunisie ) veille au respect de ces valeurs limites.
* Pour cela, elle élabore un protocole de mesure de l’exposition aux ondes et l’actualise
en fonction des évolutions technologiques. Ce protocole couvre l’ensemble des
émissions radioélectriques de 9 kHz à 300 GHz.
* En 2018, deux versions de protocoles ont successivement été en vigueur :
* La première étape du processus de mesure au‐delà de 100 kHz consiste à choisir entre
une mesure de l’exposition globale **large bande (cas A)** ou **détaillée en fréquences (cas
B)**.
* **Le cas A** fournit toutes les sources et fréquences et repose sur l’utilisation d’une sonde
large bande.
* **Le cas B** , toujours précédé d’un cas A, précise les valeurs de champs par sources,
fréquences ou sous‐bandes de fréquences. Il repose sur l’utilisation d’un analyseur de
spectre portatif.

### 3. organigramme permettant d’expliquer les processus des protocoles de mesures.

<img src="images/diagramme.PNG" />

#### Dans ce mini projet, on va supposé que les mesure dans ce dataset sont des mesures large bande (cas A) et on va essayer de déduire de maniére intelligente est ce que c'est conforme ou non au dépassement de 6v c'est à dire est ce qu'on va passer a la phase de mesure de la bande étroite (cas B) ou non. 
* pour cela on va utiliser de l'intelligence et de l'apprentissage.


## 1er phase : l'apprentissage

Pour cela, j'ai utilisé une petite dataset équilibré (en fait j'ai rajouté des lignes dont la valeur globale est supérieur à 6v). 

### Importation de modules utiles

In [4]:
%matplotlib inline
import pandas as pd 
import numpy as npGG
import matplotlib.pyplot as plt
import seaborn as sns
df=pd.read_csv('MesuresOriginalesEquilibrees.csv') #lire le fichier de mesures originales équilibrés

### Affichage des nouvelles lignes rajoutés :
Remarquons que toutes les valeurs du niveau globale sont supérieur à 6V (seuil)

In [5]:
df.tail(300)

,Numéro de mesure,Longitude,Latitude,Position,Insee,Adresse,Complément,Code postal,Commune,Date de mesure,Mesure réalisée,Rapport,Conformité de la mesure,Niveau global,Type d'environnement,Laboratoire
914,324,3.75833,43.58528,"03° 45' 30.00'' Est, 43° 35' 07.00'' Nord",34202,Ancien chemin du Murviel,Point B : chambre de la fille de Mr Esteve au ...,34570,PIGNAN,7/11/2001,En intérieur,FicheMesure324.pdf,Oui,7.10,Lieu d'habitation,BUREAU VERITAS
915,512,2.96306,43.34750,"02° 57' 47.00'' Est, 43° 20' 51.00'' Nord",34226,Rue des Ecoles,Ecole Maternelle,34310,QUARANTE,3/4/2002,En intérieur,FicheMesure512.pdf,Oui,6.80,Lieu d'habitation,APAVE
916,544,3.03167,43.36028,"03° 01' 54.00'' Est, 43° 21' 37.00'' Nord",34225,Chemin de Montfaucon,résidence de Mme SOUTADE,34620,PUISSERGUIER,23/5/2002,En intérieur,FicheMesure544.pdf,Oui,6.99,Lieu d'habitation,APAVE
917,1116,3.20806,43.34861,"03° 12' 29.00'' Est, 43° 20' 55.00'' Nord",34032,15 Rue des Cygnes,Sur le palier du dernier étage.,34500,BEZIERS,20/2/2003,En intérieur,FicheMesure1116.pdf,Oui,6.01,Lieu d'habitation,CSTB - Saint Martin d'Heres
918,1117,3.25639,43.33500,"03° 15' 23.00'' Est, 43° 20' 06.00'' Nord",34032,12 Boulevard Jean Bouin,Sur le palier du seizième et dernier étage du ...,34500,BEZIERS,20/2/2003,En intérieur,FicheMesure1117.pdf,Oui,6.10,Lieu d'habitation,CSTB - Saint Martin d'Heres
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1209,20585,3.83833,43.56333,"03° 50' 18.00'' Est, 43° 33' 48.00'' Nord",34270,Rue Saint Exupéry,"Z.I. La Lauze, Unibéton, sur la passerelle de ...",34430,SAINT-JEAN-DE-VEDAS,15/4/2010,En extérieur,FicheMesure20585.pdf,Oui,9.55,Divers,AEXPERTISE
1210,20586,3.83083,43.58361,"03° 49' 51.00'' Est, 43° 35' 01.00'' Nord",34270,Chemin des Oliviers,"Sur l'aire de jeux pour enfants, à côté de la ...",34430,SAINT-JEAN-DE-VEDAS,15/4/2010,En extérieur,FicheMesure20586.pdf,Oui,9.55,Divers,AEXPERTISE
1211,20647,3.83750,43.60556,"03° 50' 15.00'' Est, 43° 36' 20.00'' Nord",34172,Rue Viollet Le Duc,Ecole Elémentaire SPINOZA - Plateau d'évolution,34000,MONTPELLIER,24/3/2010,En extérieur,FicheMesure20647.pdf,Oui,10.02,Rue / Route / Parking,EMITECH Chassieu
1212,20649,3.91917,43.58750,"03° 55' 09.00'' Est, 43° 35' 15.00'' Nord",34129,Rue du Mas de Portal,Parking face aux maisons numéro 5 et 7,34970,LATTES,24/3/2010,En extérieur,FicheMesure20649.pdf,Oui,8.77,Rue / Route / Parking,EMITECH Chassieu


In [6]:
df.rename(columns={'Niveau global': 'Niveau_global'}, inplace=True)

### Ajout d'une nouvelle colonne appelé "Target"
dans cette colonne , les label vont être placés qui sont de valeur 0 ou 1 et ceci selon les valeurs de mesure globales (supérieur ou inferieur à 6v)
* **valeur du niveau global > 6v ------> Target = 0**
* **valeur du niveau global < 6v ------> Target = 1**

In [7]:
df=df.assign(target=1) 
print(df)

      Numéro de mesure  Longitude  Latitude  \
0                  175    3.91611  43.58528   
1                  180    4.04194  43.64694   
2                  543    3.03167  43.36028   
3                  545    3.03278  43.36167   
4                  850    3.56444  43.64583   
...                ...        ...       ...   
1209             20585    3.83833  43.56333   
1210             20586    3.83083  43.58361   
1211             20647    3.83750  43.60556   
1212             20649    3.91917  43.58750   
1213             20668    3.95944  43.57917   

                                       Position  Insee  \
0     03° 54' 58.00'' Est, 43° 35' 07.00'' Nord  34129   
1     04° 02' 31.00'' Est, 43° 38' 49.00'' Nord  34176   
2     03° 01' 54.00'' Est, 43° 21' 37.00'' Nord  34225   
3     03° 01' 58.00'' Est, 43° 21' 42.00'' Nord  34225   
4     03° 33' 52.00'' Est, 43° 38' 45.00'' Nord  34114   
...                                         ...    ...   
1209  03° 50' 18.00'' Est, 43

In [8]:
df.columns

Index(['Numéro de mesure', 'Longitude', 'Latitude', 'Position', 'Insee',
       'Adresse', 'Complément', 'Code postal', 'Commune', 'Date de mesure',
       'Mesure réalisée', 'Rapport', 'Conformité de la mesure',
       'Niveau_global', 'Type d'environnement', 'Laboratoire', 'target'],
      dtype='object')

### Définition de la fonction target :
* à travers cette fonction on va affecté la valeur 0 a tous les niveau global de valeur supérieur à 6v

In [9]:
def target():
    for t in range(0,len(df)):
        if df['Niveau_global'][t] > 6 :
            df['target'][t] = 0
            print(df['Niveau_global'][t])

* Pour afficher ses valeurs supérieur à 6v :

In [10]:
target()

<ipython-input-9-6845047a0477>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['target'][t] = 0


9.63
8.44
8.03
9.06
9.05
6.65
15.66
25.39
10.33
9.1
6.98
21.03
10.57
8.52
7.7
6.9
7.1
6.8
6.99
6.01
6.1
7.8
8.1
6.2
9.5
8.7
7.7
6.02
10.2
9.9
8.77
6.88
7.45
11.8
9.77
6.66
7.9
6.7
7.89
8.78
10.5
11.11
10.54
6.99
6.25
6.01
6.78
6.8
6.7
7.45
6.2
6.44
6.47
7.84
6.78
6.45
7.78
10.01
6.5
8.78
9.78
6.66
6.44
6.48
7.77
6.99
6.05
6.98
8.8
9.5
10.5
6.8
7.8
9.99
6.99
8.44
10.01
6.7
7.77
7.8
9.9
7.07
7.84
6.06
8.78
10.4
6.78
8.7
6.54
8.88
6.14
8.7
6.88
9.77
10.47
9.78
8.88
6.66
7.77
6.78
9.78
6.78
9.78
6.45
6.89
6.66
7.47
6.47
6.78
9.41
11.47
6.48
7.88
6.05
6.47
8.77
7.77
6.48
7.77
6.84
8.78
8.77
6.47
7.48
11.02
9.78
9.78
6.47
8.78
6.48
9.78
6.45
9.78
7.87
6.45
6.47
10.57
15.5
12.25
6.48
6.47
9.58
6.45
8.78
6.48
7.87
8.88
9.58
6.55
7.88
6.66
6.02
6.47
7.78
9.98
6.78
7.87
9.84
9.99
10.25
7.84
9.99
8.85
7.77
6.65
8.87
9.78
6.84
7.88
8.88
8.78
6.95
7.84
6.45
6.1
6.45
6.97
8.88
6.95
8.44
6.08
6.09
8.78
9.45
6.45
9.84
8.48
9.45
10.02
11.05
9.95
8.77
7.65
8.88
9.99
10.54
6.8
7.95
8.48
9.78
9.2
10.9
9.7

In [11]:
pd.crosstab(df.target, "freq") #affichage d'un petit tableau résumant le nombre total de zéro et de un

col_0,freq
target,
0,316
1,898


* Remarquons qu'on a 316 label de valeur 0 et 898 label de valeur 1

In [12]:
df.head(50)

,Numéro de mesure,Longitude,Latitude,Position,Insee,Adresse,Complément,Code postal,Commune,Date de mesure,Mesure réalisée,Rapport,Conformité de la mesure,Niveau_global,Type d'environnement,Laboratoire,target
0,175,3.91611,43.58528,"03° 54' 58.00'' Est, 43° 35' 07.00'' Nord",34129,Mas de Gau,NaN,34970,LATTES,6/12/2001,En extérieur,FicheMesure175.pdf,Oui,0.00,Divers,SATELIT,1
1,180,4.04194,43.64694,"04° 02' 31.00'' Est, 43° 38' 49.00'' Nord",34176,5539 Rue Lou Claou de la Oulivettas,NaN,34130,MUDAISON,6/12/2001,En extérieur,FicheMesure180.pdf,Oui,0.00,Divers,SATELIT,1
2,543,3.03167,43.36028,"03° 01' 54.00'' Est, 43° 21' 37.00'' Nord",34225,Chemin de Montfaucon,résidence de Mme SOUTADE,34620,PUISSERGUIER,23/5/2002,En extérieur,FicheMesure543.pdf,Oui,0.54,Divers,APAVE,1
3,545,3.03278,43.36167,"03° 01' 58.00'' Est, 43° 21' 42.00'' Nord",34225,Chemin de Quarante,résidence de M. GONZALEZ,34620,PUISSERGUIER,23/5/2002,En extérieur,FicheMesure545.pdf,Oui,0.20,Divers,APAVE,1
4,850,3.56444,43.64583,"03° 33' 52.00'' Est, 43° 38' 45.00'' Nord",34114,480 Chemin de Pioch Courbi,Chez Mme et M. GALOIS,34150,GIGNAC,10/10/2002,En extérieur,FicheMesure850.pdf,Oui,0.53,Rue / Route / Parking,APAVE,1
5,851,3.56306,43.64806,"03° 33' 47.00'' Est, 43° 38' 53.00'' Nord",34114,Vieux chemin,Chez Mme et M. HERMITTE.,34150,GIGNAC,10/10/2002,En extérieur,FicheMesure851.pdf,Oui,0.78,Rue / Route / Parking,APAVE,1
6,852,3.56306,43.64694,"03° 33' 47.00'' Est, 43° 38' 49.00'' Nord",34114,401 Chemin de Pioch Courbi,Chez Mme et M. EON.,34150,GIGNAC,10/10/2002,En extérieur,FicheMesure852.pdf,Oui,0.53,Rue / Route / Parking,APAVE,1
7,1072,3.91611,43.58500,"03° 54' 58.00'' Est, 43° 35' 06.00'' Nord",34129,Château d'eau de Boirargues,Au pied du château d'eau.,34970,LATTES,22/1/2003,En extérieur,FicheMesure1072.pdf,Oui,0.00,Rue / Route / Parking,CSTB - Saint Martin d'Heres,1
8,1073,3.91667,43.58667,"03° 55' 00.00'' Est, 43° 35' 12.00'' Nord",34129,Avenue de la Mer - Garage Tunning XXS,Au début du chemin donnant sur le château d'ea...,34970,LATTES,22/1/2003,En extérieur,FicheMesure1073.pdf,Oui,0.54,Rue / Route / Parking,CSTB - Saint Martin d'Heres,1
9,1074,3.91889,43.58583,"03° 55' 08.00'' Est, 43° 35' 09.00'' Nord",34129,Chemin des Mas de Gau,Au début de la piste cyclable donnant sur le c...,34970,LATTES,22/1/2003,En extérieur,FicheMesure1074.pdf,Oui,0.00,Rue / Route / Parking,CSTB - Saint Martin d'Heres,1


In [13]:
X=df[['Numéro de mesure','Niveau_global']].to_numpy()
print(X)
y=df['target'].to_numpy()
print(y)

[[1.7500e+02 0.0000e+00]
 [1.8000e+02 0.0000e+00]
 [5.4300e+02 5.4000e-01]
 ...
 [2.0647e+04 1.0020e+01]
 [2.0649e+04 8.7700e+00]
 [2.0668e+04 7.7700e+00]]
[1 1 1 ... 0 0 0]


## <font color="red"> La phase d'apprentissage

* Dans cette phase j'ai divisé les valeurs de x en X_train et X_test , de même pour les valeurs de y.
* J'ai choisit de prendre 70% des lignes pour l'entrainement et 30% pour le test
* J'ai choisit comme modèle d'apprentissage de machine learning le "Logistic Regression"

In [14]:
#pour l'apprentissage on a diviser x en X_train et X_test et y en y_train et y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

## **Test size = 365 ça represente le 30% de 1214 lignes (nombre totale des lignes du dataset)**

In [15]:
from sklearn.linear_model import LogisticRegression 
#Create a l.Reg Classifier
logreg1 = LogisticRegression()
#Train the model using the training sets
logreg1.fit(X_train, y_train)
y_pred=logreg1.predict(X_test)

### Affichage de la matrice de confusion 

In [33]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[109   0]
 [  2 254]]
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       109
           1       1.00      0.99      1.00       256

    accuracy                           0.99       365
   macro avg       0.99      1.00      0.99       365
weighted avg       0.99      0.99      0.99       365

0.9945205479452055


##### **Remarquons que toutes les label de valeur 0 sont prédites correctement et on a 2 label de valeur 1 sont prédites d'une façon irroné et 254 label de valeur 1 sont prédites correctement.**
* Le score est presque égale à 100%

## <font color="red">Test du modèle avec des nouvelles valeurs

#### Dans ce stade, pour le test j'ai réaliser un nouveau dataset, nommé : "NouvellesMesure"

In [34]:
df1=pd.read_csv('NouvellesMesure.csv') #lire le fichier de nouvelles mesures 

In [5]:
df.head(50)

,Numéro de mesure,Longitude,Latitude,Position,Insee,Adresse,Complément,Code postal,Commune,Date de mesure,Mesure réalisée,Rapport,Conformité de la mesure,Niveau global,Type d'environnement,Laboratoire
0,175,3.91611,43.58528,"03° 54' 58.00'' Est, 43° 35' 07.00'' Nord",34129,Mas de Gau,NaN,34970,LATTES,6/12/2001,En extérieur,FicheMesure175.pdf,Oui,0.00,Divers,SATELIT
1,180,4.04194,43.64694,"04° 02' 31.00'' Est, 43° 38' 49.00'' Nord",34176,5539 Rue Lou Claou de la Oulivettas,NaN,34130,MUDAISON,6/12/2001,En extérieur,FicheMesure180.pdf,Oui,0.00,Divers,SATELIT
2,543,3.03167,43.36028,"03° 01' 54.00'' Est, 43° 21' 37.00'' Nord",34225,Chemin de Montfaucon,résidence de Mme SOUTADE,34620,PUISSERGUIER,23/5/2002,En extérieur,FicheMesure543.pdf,Oui,0.54,Divers,APAVE
3,545,3.03278,43.36167,"03° 01' 58.00'' Est, 43° 21' 42.00'' Nord",34225,Chemin de Quarante,résidence de M. GONZALEZ,34620,PUISSERGUIER,23/5/2002,En extérieur,FicheMesure545.pdf,Oui,0.20,Divers,APAVE
4,850,3.56444,43.64583,"03° 33' 52.00'' Est, 43° 38' 45.00'' Nord",34114,480 Chemin de Pioch Courbi,Chez Mme et M. GALOIS,34150,GIGNAC,10/10/2002,En extérieur,FicheMesure850.pdf,Oui,0.53,Rue / Route / Parking,APAVE
5,851,3.56306,43.64806,"03° 33' 47.00'' Est, 43° 38' 53.00'' Nord",34114,Vieux chemin,Chez Mme et M. HERMITTE.,34150,GIGNAC,10/10/2002,En extérieur,FicheMesure851.pdf,Oui,0.78,Rue / Route / Parking,APAVE
6,852,3.56306,43.64694,"03° 33' 47.00'' Est, 43° 38' 49.00'' Nord",34114,401 Chemin de Pioch Courbi,Chez Mme et M. EON.,34150,GIGNAC,10/10/2002,En extérieur,FicheMesure852.pdf,Oui,0.53,Rue / Route / Parking,APAVE
7,1072,3.91611,43.58500,"03° 54' 58.00'' Est, 43° 35' 06.00'' Nord",34129,Château d'eau de Boirargues,Au pied du château d'eau.,34970,LATTES,22/1/2003,En extérieur,FicheMesure1072.pdf,Oui,0.00,Rue / Route / Parking,CSTB - Saint Martin d'Heres
8,1073,3.91667,43.58667,"03° 55' 00.00'' Est, 43° 35' 12.00'' Nord",34129,Avenue de la Mer - Garage Tunning XXS,Au début du chemin donnant sur le château d'ea...,34970,LATTES,22/1/2003,En extérieur,FicheMesure1073.pdf,Oui,0.54,Rue / Route / Parking,CSTB - Saint Martin d'Heres
9,1074,3.91889,43.58583,"03° 55' 08.00'' Est, 43° 35' 09.00'' Nord",34129,Chemin des Mas de Gau,Au début de la piste cyclable donnant sur le c...,34970,LATTES,22/1/2003,En extérieur,FicheMesure1074.pdf,Oui,0.00,Rue / Route / Parking,CSTB - Saint Martin d'Heres


In [35]:
df1.rename(columns={'Niveau global': 'Niveau_global'}, inplace=True)

In [36]:
df1=df1.assign(target1=1) #création de la colonne des label
print(df1)

    Numéro de mesure  Longitude  Latitude  \
0               8495    3.85333  43.59111   
1               8496    3.85417  43.59083   
2               8497    3.85333  43.59028   
3               8503    3.83861  43.56361   
4               8505    3.83861  43.56361   
5               9167    3.87806  43.60556   
6               9168    3.87806  43.60556   
7               9169    3.87806  43.60556   
8               9524    3.82389  43.63361   
9               9529    3.94556  43.57028   
10             10061    3.83944  43.61694   
11             10062    3.83944  43.61611   
12             10063    3.86361  43.59278   
13             10064    3.86500  43.59306   
14             10148    3.60750  43.42583   
15             10669    3.88083  43.61611   
16             10675    3.73889  43.79167   
17             10676    3.74028  43.79250   
18             11186    3.82778  43.61361   
19             11187    3.82944  43.61139   
20             11188    3.82972  43.61194   
21        

<font size="3"><font color="green">Remarquons qu'on a 50 lignes 
et on a 16 colonnes initiale plus la colonne target ajoutée recemment

In [37]:
def target1(): #Si le niveau globale>6 on va avoir target=0
    for t in range(0,len(df1)):
        if df1['Niveau_global'][t] > 6 :
            df1['target1'][t] = 0
            print(df1['Niveau_global'][t])

In [38]:
target1 ()

11.5
6.99
9.02
6.99
7.66
8.22
8.65
7.69
8.01
7.65
6.66
6.78
7.89
6.85
9.26
8.65
9.25
9.84
8.99
6.88
7.06
8.03
6.09
9.33
7.58
9.66


<ipython-input-37-1bb125fa1923>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['target1'][t] = 0


<font size="3"><font color="green">Remarquons qu'il y a 26 valeurs du niveau global supérieur à 6

In [39]:
pd.crosstab(df1.target1, "freq")

col_0,freq
target1,
0,26
1,24


<font size="3"><font color="F64B91">**Dans le nouveau dataset j'ai 26 label de valeur 0 et 24 label de valeur 1**

In [40]:
Z=df1[['Numéro de mesure','Niveau_global']].to_numpy()
print(Z)
z=df1['target1'].to_numpy()
print(z)

[[8.49500e+03 4.20000e-01]
 [8.49600e+03 4.00000e-01]
 [8.49700e+03 1.90000e-01]
 [8.50300e+03 2.68000e+00]
 [8.50500e+03 5.99000e+00]
 [9.16700e+03 5.05000e+00]
 [9.16800e+03 2.20000e-01]
 [9.16900e+03 2.50000e-01]
 [9.52400e+03 2.30000e-01]
 [9.52900e+03 1.52000e+00]
 [1.00610e+04 1.30000e+00]
 [1.00620e+04 6.00000e+00]
 [1.00630e+04 5.89000e+00]
 [1.00640e+04 0.00000e+00]
 [1.01480e+04 5.80000e-01]
 [1.06690e+04 0.00000e+00]
 [1.06750e+04 3.70000e-01]
 [1.06760e+04 0.00000e+00]
 [1.11860e+04 2.47000e+00]
 [1.11870e+04 2.80000e+00]
 [1.11880e+04 5.90000e+00]
 [1.16640e+04 6.90000e-01]
 [1.18130e+04 4.90000e-01]
 [1.18140e+04 0.00000e+00]
 [2.27330e+04 1.15000e+01]
 [1.14611e+05 6.99000e+00]
 [1.14624e+05 9.02000e+00]
 [1.14903e+05 6.99000e+00]
 [1.14925e+05 7.66000e+00]
 [1.15133e+05 8.22000e+00]
 [1.15225e+05 8.65000e+00]
 [1.15853e+05 7.69000e+00]
 [1.15857e+05 8.01000e+00]
 [1.15859e+05 7.65000e+00]
 [1.15863e+05 6.66000e+00]
 [1.16139e+05 6.78000e+00]
 [1.17591e+05 7.89000e+00]
 

In [41]:
y1_pred=logreg1.predict(Z)
print(y1_pred)

[1 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [42]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(z,y1_pred))
print(classification_report(z,y1_pred))
print(accuracy_score(z, y1_pred))

[[26  0]
 [ 4 20]]
              precision    recall  f1-score   support

           0       0.87      1.00      0.93        26
           1       1.00      0.83      0.91        24

    accuracy                           0.92        50
   macro avg       0.93      0.92      0.92        50
weighted avg       0.93      0.92      0.92        50

0.92


## * D'aprés la matrice de confusion remarquons:  que toutes les zéro sont prédit correctement, c'est logique ! puisque les valeur dans le dataset les valeurs inférieur à 6v sont loin du seuil qui est égale à 6v.
## * et que 4 label de valeur 1 (correspondant au niveau global inférieur à 6) sont prédit d'une façon irronée et 20 uns sont prédit correctement.**
## * Et on a une précision de 92%**